In [ ]:
from cdcqr.data.glassnode.utils import get_glassnode_data
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import numpy as np
import vectorbt as vbt
import seaborn as sns

### test

In [ ]:
API_KEY = '1wvlrc0JsEMIWF6enI69cWiOojq'

catagory = 'transactions'
feature =  'transfers_volume_to_exchanges_mean'
underly = 'BTC'
freq = '10m'
start_time = dt.now()-timedelta(hours=1000)

In [ ]:
df = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY)

In [ ]:
dfbtc = pd.read_pickle('/tmp/btc_price_20211001_20211205_10Min.pkl')
dfeth = pd.read_pickle('/tmp/btc_price_20211001_20211205_10Min.pkl')

In [ ]:
common_index = df.index.intersection(dfbtc.index)

In [ ]:
dfbtc = dfbtc.reindex(index=common_index)
dfeth = dfeth.reindex(index=common_index)

In [ ]:
def quantile_info(dff):
    q01 = dff.quantile(0.01).values[0]
    q02 = dff.quantile(0.02).values[0]
    q05 = dff.quantile(0.05).values[0]
    q95 = dff.quantile(0.95).values[0]
    q98 = dff.quantile(0.98).values[0]
    q99 = dff.quantile(0.99).values[0]
    print(q01, q02, q05, q95, q98, q99)
    return q01, q02, q05, q95, q98, q99

def signal_f1(x):
    if x>=THRESHOLD:
        return 1
    elif x<-THRESHOLD:
        return -1
    else:
        return 0
    
def signal_f2(x):
    if x>=THRESHOLD:
        return -1
    elif x<-THRESHOLD:
        return 1
    else:
        return 0

### feature 1) 'transfers_volume_to_exchanges_mean'

In [ ]:
feature = 'transfers_volume_to_exchanges_mean'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

In [ ]:
_ = quantile_info(dff)

In [ ]:
THRESHOLD1 = 0.9 
THRESHOLD2 = 2.5
n = 20
lag_dict = {}
for lag in range(0, 5, 1):
    threshold_dict = {}
    for THRESHOLD in np.linspace(THRESHOLD1, THRESHOLD2, n):
        btc_signal = dff['v'].apply(signal_f1)
        trade = btc_signal.replace(0, np.nan).ffill().fillna(0).diff().shift(lag)
        entries = trade.copy()
        entries = trade>0
        exits = trade<0
        entries1 = entries.reindex(dfbtc.index)
        exits1 = exits.reindex(dfbtc.index)
        pf_kwargs = dict(fees=0.0007, freq='1Min')
        pf = vbt.Portfolio.from_signals(dfbtc['o'], entries1, exits1, init_cash=100, **pf_kwargs)
        threshold_dict[THRESHOLD] = pf.sharpe_ratio()
    lag_dict[lag] = threshold_dict

pd.DataFrame(lag_dict).to_pickle('{}.pkl'.format(feature))

### transfers_volume_from_exchanges_mean

In [ ]:
feature = 'transfers_volume_from_exchanges_mean'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

_ = quantile_info(dff)

In [ ]:
THRESHOLD1 = 1.5
THRESHOLD2 = 5
n = 20
lag_dict = {}
for lag in range(0, 5, 1):
    threshold_dict = {}
    for THRESHOLD in np.linspace(THRESHOLD1, THRESHOLD2, n):
        btc_signal = dff['v'].apply(signal_f1)
        trade = btc_signal.replace(0, np.nan).ffill().fillna(0).diff().shift(lag)
        entries = trade.copy()
        entries = trade>0
        exits = trade<0
        entries1 = entries.reindex(dfbtc.index)
        exits1 = exits.reindex(dfbtc.index)
        pf_kwargs = dict(fees=0.0007, freq='1Min')
        pf = vbt.Portfolio.from_signals(dfbtc['o'], entries1, exits1, init_cash=100, **pf_kwargs)
        threshold_dict[THRESHOLD] = pf.sharpe_ratio()
    lag_dict[lag] = threshold_dict

pd.DataFrame(lag_dict).to_pickle('{}.pkl'.format(feature))

### TransfersFromExchangesCount

In [ ]:
feature = 'transfers_from_exchanges_count'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

_ = quantile_info(dff)

In [ ]:
THRESHOLD1 = 800
THRESHOLD2 = 1300
n = 20
lag_dict = {}
for lag in range(0, 5, 1):
    threshold_dict = {}
    for THRESHOLD in np.linspace(THRESHOLD1, THRESHOLD2, n):
        btc_signal = dff['v'].apply(signal_f1)
        trade = btc_signal.replace(0, np.nan).ffill().fillna(0).diff().shift(lag)
        entries = trade.copy()
        entries = trade>0
        exits = trade<0
        entries1 = entries.reindex(dfbtc.index)
        exits1 = exits.reindex(dfbtc.index)
        pf_kwargs = dict(fees=0.0007, freq='1Min')
        pf = vbt.Portfolio.from_signals(dfbtc['o'], entries1, exits1, init_cash=100, **pf_kwargs)
        threshold_dict[THRESHOLD] = pf.sharpe_ratio()
    lag_dict[lag] = threshold_dict

pd.DataFrame(lag_dict).to_pickle('{}.pkl'.format(feature))

### TransfersVolumeMinersNet

In [ ]:
feature = 'transfers_volume_miners_net'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

_ = quantile_info(dff)

In [ ]:
THRESHOLD1 = 10
THRESHOLD2 = 70
n = 20
lag_dict = {}
for lag in range(0, 5, 1):
    threshold_dict = {}
    for THRESHOLD in np.linspace(THRESHOLD1, THRESHOLD2, n):
        btc_signal = dff['v'].apply(signal_f1)
        trade = btc_signal.replace(0, np.nan).ffill().fillna(0).diff().shift(lag)
        entries = trade.copy()
        entries = trade>0
        exits = trade<0
        entries1 = entries.reindex(dfbtc.index)
        exits1 = exits.reindex(dfbtc.index)
        pf_kwargs = dict(fees=0.0007, freq='1Min')
        pf = vbt.Portfolio.from_signals(dfbtc['o'], entries1, exits1, init_cash=100, **pf_kwargs)
        threshold_dict[THRESHOLD] = pf.sharpe_ratio()
    lag_dict[lag] = threshold_dict

pd.DataFrame(lag_dict).to_pickle('{}.pkl'.format(feature))

### TransfersVolumeSum

In [ ]:
feature = 'transfers_volume_sum'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

_ = quantile_info(dff)

### TransfersVolumeProfitRelative

In [ ]:
feature = 'transfers_volume_profit_relative'
underly = 'BTC'
dff = get_glassnode_data(catagory, feature, underly, freq, start_time, API_KEY).reindex(index=common_index)

_ = quantile_info(dff)